In [139]:
import pandas as pd
import os
import numpy as np

from tbparse import SummaryReader

In [140]:
os.getcwd()
# change directory to assembly_glovebox_dataet 
os.chdir('/home/shivansh/repos/assembly_glovebox_dataset')

In [141]:
versions = {
    'unet': {
        'ood': 'version_153',
        'ood, dropout': 'version_154',
        'id': 'version_155',
        'id, dropout': 'version_156',
        'replaced_gs, ood': 'version_161',
        'replaced_gs, ood, dropout': 'version_162',
        'replaced_gs': 'version_163',
        'replaced_gs, dropout': 'version_164'
    },
    'bisenet': {
        'ood': 'version_149',
        'ood, dropout': 'version_150',
        'id': 'version_151',
        'id, dropout': 'version_152',
        'replaced_gs, ood': 'version_165',
        'replaced_gs, ood, dropout': 'version_166',
        'replaced_gs': 'version_167',
        'replaced_gs, dropout': 'version_168'
    },
    'mobilesam': {
        'ood': 'version_174',
        'ood, dropout': 'version_175',
        'id': 'version_176',
        'id, dropout': 'version_177',
        'replaced_gs, ood': 'version_178',
        'replaced_gs, ood, dropout': 'version_179',
        'replaced_gs': 'version_180',
        'replaced_gs, dropout': 'version_181'
    },
    'unet_ensemble': {
        'ood': 'ood_unet_dropout_False/version_0',
        'ood, dropout': 'ood_unet_dropout_True/version_0',
        'id': 'id_unet_dropout_False/version_0',
        'id, dropout': 'id_unet_dropout_True/version_0',
        'replaced_gs, ood': 'ood+gs_unet_dropout_False/version_0',
        'replaced_gs, ood, dropout': 'ood+gs_unet_dropout_True/version_0',
        'replaced_gs': 'gs_unet_dropout_False/version_0',
        'replaced_gs, dropout': 'gs_unet_dropout_True/version_0'
    },
    'bisenet_ensemble': {
        'ood': 'ood_bisenetv2_dropout_False/version_0',
        'ood, dropout': 'ood_bisenetv2_dropout_True/version_0',
        'id': 'id_bisenetv2_dropout_False/version_0',
        'id, dropout': 'id_bisenetv2_dropout_True/version_0',
        'replaced_gs, ood': 'ood+gs_bisenetv2_dropout_False/version_0',
        'replaced_gs, ood, dropout': 'ood+gs_bisenetv2_dropout_True/version_0',
        'replaced_gs': 'gs_bisenetv2_dropout_False/version_0',
        'replaced_gs, dropout': 'gs_bisenetv2_dropout_True/version_0'
    },
    'mobilesam_ensemble': {
        'ood': 'ood_mobilesam_dropout_False/version_0',
        'ood, dropout': 'ood_mobilesam_dropout_True/version_0',
        'id': 'id_mobilesam_dropout_False/version_0',
        'id, dropout': 'id_mobilesam_dropout_True/version_0',
        'replaced_gs, ood': 'ood+gs_mobilesam_dropout_False/version_0',
        'replaced_gs, ood, dropout': 'ood+gs_mobilesam_dropout_True/version_0',
        'replaced_gs': 'gs_mobilesam_dropout_False/version_0',
        'replaced_gs, dropout': 'gs_mobilesam_dropout_True/version_0'
    }

}

In [142]:
# create pandas dataframe

dfs = []

for model in versions:
    for set in versions[model]:
        version = versions[model][set]
        log_dir = f"./lightning_logs/{version}"
        reader =  SummaryReader(log_dir)
        df = reader.scalars

        print(model, set, len(df))

        # TODO: rename respective versions to type with dictionary
        df['type'] = set
        df['model'] = model

        dfs.append(df[~df['tag'].isin(['epoch', 'test_loss'])])

result = pd.concat(dfs)

unet ood 5
unet ood, dropout 5
unet id 5
unet id, dropout 5
unet replaced_gs, ood 5
unet replaced_gs, ood, dropout 5
unet replaced_gs 5
unet replaced_gs, dropout 5
bisenet ood 5
bisenet ood, dropout 5
bisenet id 5
bisenet id, dropout 5


bisenet replaced_gs, ood 5
bisenet replaced_gs, ood, dropout 5
bisenet replaced_gs 5
bisenet replaced_gs, dropout 5
mobilesam ood 5
mobilesam ood, dropout 5
mobilesam id 5
mobilesam id, dropout 5
mobilesam replaced_gs, ood 5
mobilesam replaced_gs, ood, dropout 5
mobilesam replaced_gs 5
mobilesam replaced_gs, dropout 5
unet_ensemble ood 5
unet_ensemble ood, dropout 5
unet_ensemble id 5
unet_ensemble id, dropout 5
unet_ensemble replaced_gs, ood 5
unet_ensemble replaced_gs, ood, dropout 5
unet_ensemble replaced_gs 5
unet_ensemble replaced_gs, dropout 5
bisenet_ensemble ood 5
bisenet_ensemble ood, dropout 5
bisenet_ensemble id 5
bisenet_ensemble id, dropout 5
bisenet_ensemble replaced_gs, ood 5
bisenet_ensemble replaced_gs, ood, dropout 5
bisenet_ensemble replaced_gs 5
bisenet_ensemble replaced_gs, dropout 5
mobilesam_ensemble ood 5
mobilesam_ensemble ood, dropout 5
mobilesam_ensemble id 5
mobilesam_ensemble id, dropout 5
mobilesam_ensemble replaced_gs, ood 5
mobilesam_ensemble replaced_gs

In [143]:
result

,step,tag,value,type,model
0,0,avg_frame_reference_time,0.004106,ood,unet
2,0,test_ace,10.057115,ood,unet
3,0,test_iou,0.570464,ood,unet
0,0,avg_frame_reference_time,0.004627,"ood, dropout",unet
2,0,test_ace,10.047482,"ood, dropout",unet
...,...,...,...,...,...
2,0,test_ace,2.566478,replaced_gs,mobilesam_ensemble
3,0,test_iou,0.489686,replaced_gs,mobilesam_ensemble
0,0,avg_frame_reference_time,0.046183,"replaced_gs, dropout",mobilesam_ensemble
2,0,test_ace,2.586055,"replaced_gs, dropout",mobilesam_ensemble


In [144]:
dfs = {value: result.loc[result['model'] == value] for value in ['bisenet', 'mobilesam', 'unet', 'bisenet_ensemble', 'mobilesam_ensemble', 'unet_ensemble']}

In [145]:
x = dfs['mobilesam'].merge(dfs['bisenet'], on=['tag', 'type'], suffixes=("_mobilesam","_bisenet")).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag']))
x = x.merge(dfs['unet'], on=['tag', 'type']).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag', 'step']))
x = x.merge(dfs['bisenet_ensemble'], on=['tag', 'type'], suffixes=("_unet","_bisenet_ensemble")).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag', 'step']))
x = x.merge(dfs['mobilesam_ensemble'], on=['tag', 'type']).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag', 'step']))
x = x.merge(dfs['unet_ensemble'], on=['tag', 'type'], suffixes=("_mobilesam_ensemble","_unet_ensemble")).groupby(["tag"], group_keys=True).apply(lambda x: x.drop(columns=['tag', 'step']))
x

step_mobilesam  value_mobilesam  \
tag                                                            
avg_frame_reference_time 0                0         0.032603   
                         1                0         0.035590   
                         2                0         0.052599   
                         3                0         0.053561   
                         4                0         0.022766   
                         5                0         0.024426   
                         6                0         0.032336   
                         7                0         0.034081   
test_ace                 8                0         8.790260   
                         9                0         8.756591   
                         10               0         6.458765   
                         11               0         6.440013   
                         12               0         9.989725   
                         13               0         9.970565   
                         14               0         8.742709   
                         15               0         8.744898   
test_iou                 16               0         0.529014   
                         17               0         0.503160   
                         18               0         0.572871   
                         19               0         0.554973   
                         20               0         0.509744   
                         21               0         0.509428   
                         22               0         0.515979   
                         23               0         0.532306   

                                                  type model_mobilesam  \
tag                                                                      
avg_frame_reference_time 0                         ood       mobilesam   
                         1                ood, dropout       mobilesam   
                         2                          id       mobilesam   
                         3                 id, dropout       mobilesam   
                         4            replaced_gs, ood       mobilesam   
                         5   replaced_gs, ood, dropout       mobilesam   
                         6                 replaced_gs       mobilesam   
                         7        replaced_gs, dropout       mobilesam   
test_ace                 8                         ood       mobilesam   
                         9                ood, dropout       mobilesam   
                         10                         id       mobilesam   
                         11                id, dropout       mobilesam   
                         12           replaced_gs, ood       mobilesam   
                         13  replaced_gs, ood, dropout       mobilesam   
                         14                replaced_gs       mobilesam   
                         15       replaced_gs, dropout       mobilesam   
test_iou                 16                        ood       mobilesam   
                         17               ood, dropout       mobilesam   
                         18                         id       mobilesam   
                         19                id, dropout       mobilesam   
                         20           replaced_gs, ood       mobilesam   
                         21  replaced_gs, ood, dropout       mobilesam   
                         22                replaced_gs       mobilesam   
                         23       replaced_gs, dropout       mobilesam   

                             step_bisenet  value_bisenet model_bisenet  \
tag                                                                      
avg_frame_reference_time 0              0       0.006823       bisenet   
                         1              0       0.007532       bisenet   
                         2              0       0.013415       bisenet   
                         3              0       0.014159       bisene

In [146]:
x = x.rename(columns={'value_mobilesam': 'mobilesam', 
                      'value_bisenet': 'bisenet',
                        'value_unet': 'unet',
                        'value_bisenet_ensemble': 'bisenet_ensemble',
                        'value_mobilesam_ensemble': 'mobilesam_ensemble',
                        'value_unet_ensemble': 'unet_ensemble'})


x.drop(columns=['model_unet', 'model_bisenet', 'model_mobilesam', 'model_bisenet_ensemble',
                'model_unet_ensemble', 'model_mobilesam_ensemble'], inplace=True)
# columns_to_drop = ['model_x', 'model_y', 'model']
# df = df.drop(columns=columns_to_drop)

# TODO: remove epoch columns
# TODO: fix avg time reference values

In [147]:
x

step_mobilesam  mobilesam  \
tag                                                      
avg_frame_reference_time 0                0   0.032603   
                         1                0   0.035590   
                         2                0   0.052599   
                         3                0   0.053561   
                         4                0   0.022766   
                         5                0   0.024426   
                         6                0   0.032336   
                         7                0   0.034081   
test_ace                 8                0   8.790260   
                         9                0   8.756591   
                         10               0   6.458765   
                         11               0   6.440013   
                         12               0   9.989725   
                         13               0   9.970565   
                         14               0   8.742709   
                         15               0   8.744898   
test_iou                 16               0   0.529014   
                         17               0   0.503160   
                         18               0   0.572871   
                         19               0   0.554973   
                         20               0   0.509744   
                         21               0   0.509428   
                         22               0   0.515979   
                         23               0   0.532306   

                                                  type  step_bisenet  \
tag                                                                    
avg_frame_reference_time 0                         ood             0   
                         1                ood, dropout             0   
                         2                          id             0   
                         3                 id, dropout             0   
                         4            replaced_gs, ood             0   
                         5   replaced_gs, ood, dropout             0   
                         6                 replaced_gs             0   
                         7        replaced_gs, dropout             0   
test_ace                 8                         ood             0   
                         9                ood, dropout             0   
                         10                         id             0   
                         11                id, dropout             0   
                         12           replaced_gs, ood             0   
                         13  replaced_gs, ood, dropout             0   
                         14                replaced_gs             0   
                         15       replaced_gs, dropout             0   
test_iou                 16                        ood             0   
                         17               ood, dropout             0   
                         18                         id             0   
                         19                id, dropout             0   
                         20           replaced_gs, ood             0   
                         21  replaced_gs, ood, dropout             0   
                         22                replaced_gs             0   
                         23       replaced_gs, dropout             0   

                               bisenet       unet  bisenet_ensemble  \
tag                                                                   
avg_frame_reference_time 0    0.006823   0.004106          0.004810   
                         1    0.007532   0.004627          0.000252   
                         2    0.013415   0.007269          0.000767   
                         3    0.014159   0.007681          0.000253   
                         4    0.004507   0.003354          0.002189   
                         5    0.004763   0.003556          0.000729   
                         6    0.010148   0.006386          0.005970   
        

In [148]:
x = x[['type', 'bisenet', 'mobilesam', 'unet', 'bisenet_ensemble', 'mobilesam_ensemble', 'unet_ensemble']]

x

type    bisenet  mobilesam  \
tag                                                                            
avg_frame_reference_time 0                         ood   0.006823   0.032603   
                         1                ood, dropout   0.007532   0.035590   
                         2                          id   0.013415   0.052599   
                         3                 id, dropout   0.014159   0.053561   
                         4            replaced_gs, ood   0.004507   0.022766   
                         5   replaced_gs, ood, dropout   0.004763   0.024426   
                         6                 replaced_gs   0.010148   0.032336   
                         7        replaced_gs, dropout   0.011199   0.034081   
test_ace                 8                         ood  19.734829   8.790260   
                         9                ood, dropout  19.708696   8.756591   
                         10                         id   9.925502   6.458765   
                         11                id, dropout   9.928941   6.440013   
                         12           replaced_gs, ood  17.835138   9.989725   
                         13  replaced_gs, ood, dropout  17.823523   9.970565   
                         14                replaced_gs  13.221034   8.742709   
                         15       replaced_gs, dropout  13.219630   8.744898   
test_iou                 16                        ood   0.541172   0.529014   
                         17               ood, dropout   0.548948   0.503160   
                         18                         id   0.737037   0.572871   
                         19                id, dropout   0.728564   0.554973   
                         20           replaced_gs, ood   0.529273   0.509744   
                         21  replaced_gs, ood, dropout   0.539117   0.509428   
                         22                replaced_gs   0.540659   0.515979   
                         23       replaced_gs, dropout   0.537827   0.532306   

                                  unet  bisenet_ensemble  mobilesam_ensemble  \
tag                                                                            
avg_frame_reference_time 0    0.004106          0.004810            0.049316   
                         1    0.004627          0.000252            0.045456   
                         2    0.007269          0.000767            0.045494   
                         3    0.007681          0.000253            0.045432   
                         4    0.003354          0.002189            0.045702   
                         5    0.003556          0.000729            0.045534   
                         6    0.006386          0.005970            0.046450   
                         7    0.006523          0.001881            0.046183   
test_ace                 8   10.057115         19.858597            9.782069   
                         9   10.047482         19.864529            9.786345   
                         10   9.980969         19.862703            9.685975   
                         11   9.977846         19.862614            9.677526   
                         12  10.341810          6.634458            6.450114   
                         13  10.330724          6.634404            6.425628   
                         14   9.568053         18.028381            2.566478   
                         15   9.556040          2.650289            2.586055   
test_iou                 16   0.570464          0.555689            0.505737   
                         17   0.573503          0.556784            0.505508   
                         18   0.799321          0.734263            0.519592   
                         19   0.784424          0.738399            0.492373   
                         20   0.568530          0.559196            0.517426   
                         21   0.566153          0.549275            0.474434   
                         22   0.550574          0

In [162]:
save_df = x.T

save_df.to_csv('results.csv')

# Reading Predictive Entropy Data

In [158]:
os.getcwd()

'/home/shivansh/repos/assembly_glovebox_dataset'

In [150]:
set

'replaced_gs, dropout'

In [151]:
# writing out the mean predictive entropy just to get a good glance, box plots should be easy

# create pandas dataframe
import os
import glob


columns = list(versions.keys())
index = list(versions['unet'].keys())
entropy_df = pd.DataFrame(index = index, columns=columns)

for model in versions:
    for set in versions[model]:
        version = versions[model][set]
        csv = f"./lightning_logs/{version}/raw_entropy.csv"

        df = pd.read_csv(csv, header=None)

        mean = df.mean()


        entropy_df.at[set, model] = mean.values[0]

# result = pd.concat(dfs)

In [154]:
entropy_df

,unet,bisenet,mobilesam,unet_ensemble,bisenet_ensemble,mobilesam_ensemble
ood,0.003659,0.006411,0.979398,0.003816,0.00584,0.979822
"ood, dropout",0.005112,0.007935,0.979666,0.003514,0.005624,0.979754
id,0.007556,0.008496,0.979463,0.005857,0.008323,0.979833
"id, dropout",0.007909,0.009299,1.077969,0.00602,0.008277,0.979891
"replaced_gs, ood",0.003802,0.008012,0.979439,0.004268,0.00632,0.980353
"replaced_gs, ood, dropout",0.005356,0.008667,0.979725,0.004103,0.006451,0.980199
replaced_gs,0.00421,0.0106,0.979946,0.004479,0.007451,0.980768
"replaced_gs, dropout",0.006004,0.009982,0.980029,0.004389,0.007718,0.980786


In [165]:
entropy_df.T.to_csv('entropy_results.csv')

## Next

- ensure all experiments are accurate 
- 

Data fixes
- ensemble times are lower than regular

Analysis
- make box plots for entropy data

Data observations
- MobileSAM IoU lower when ensembled, all teh rest higher

OOD vs ID 

Green Screen
- still does similar compared to OOD despite change in background

ACE
- barely any difference between regular and with dropout added
- goes way down with replaced GS, (TODO: check why bisenet here is still o high)

Time
- unet fastest, then bisenet, (TODO: rerecord bisenet)

IoU
- highest for ID
- barely any fluctuation for MobileSAM, but it is low to begin with
- replaced gs still around the same as the OOD test set
- UNet does the best, then bisenet, then lowest is mobile sam